In [1]:
import pandas as pd
import numpy as np

In [2]:
from bs4 import BeautifulSoup as b
import urllib as url #if you are using python3+ version, import urllib.request
import requests
from urllib.request import urlopen as uReq

### LNR

In [12]:
fixe = "https://www.lnr.fr/"
# acces aux pages des saisons
saison_wiki = [
    "/rugby-top-14/calendrier-resultats-rugby-top-14?season=14524&day=all",
    "/rugby-top-14/calendrier-resultats-rugby-top-14?season=14523&day=all",
    "/rugby-top-14/calendrier-resultats-rugby-top-14?season=14522&day=all",
    "/rugby-top-14/calendrier-resultats-rugby-top-14?season=14519&day=all",
    "/rugby-top-14/calendrier-resultats-rugby-top-14?season=14520&day=all",
    "/rugby-top-14/calendrier-resultats-rugby-top-14?season=14521&day=all",
    "/rugby-top-14/calendrier-resultats-rugby-top-14?season=14528&day=all",
    "/rugby-top-14/calendrier-resultats-rugby-top-14?season=14529&day=all",
    "/rugby-top-14/calendrier-resultats-rugby-top-14?season=14530&day=all",
    "/rugby-top-14/calendrier-resultats-rugby-top-14?season=14534&day=all",
    "/rugby-top-14/calendrier-resultats-rugby-top-14?season=14535&day=all",
    "/rugby-top-14/calendrier-resultats-rugby-top-14?season=18505&day=all",
    "/rugby-top-14/calendrier-resultats-rugby-top-14?season=21642&day=all",
    "/rugby-top-14/calendrier-resultats-rugby-top-14?season=24725&day=all",
    "/rugby-top-14/calendrier-resultats-rugby-top-14?season=27591&day=all"
    ]

#### saison pipeline

In [24]:
# Saison est la variable globale qui contient toute les informations de toutes les saisons
saison = []
for w in saison_wiki:
    requete = requests.get(fixe +w)
    page = requete.content
    soup = b(page)

    wiki_temp = []
    journee_name = []
    for i in (soup.findAll("section", {"class": "block block-lnr-custom block-lnr-custom-calendar-results-filter"})[0].
         findAll("span", {"class": "field-content"}))[15:]:
        wiki_temp.append(i.a["href"])
        journee_name.append(i.a["data-title"])

    journee_n = 0
    for j in wiki_temp:
        requete = requests.get(fixe+j)
        page = requete.content
        soup = b(page)
        container = soup.findAll("div", {"class":"day-results-table"})

        journee = []
        for match in container[0].findAll("tr",{"class": 'info-line after'}):
            list_info = [i.text for i in (match.findAll("span",{"format-full"}))[:-1]]
            list_score = match.find("td",{"cell-score"}).text.strip().split("-")
            list_score.append((match.find("td",{"cell-bonus-a"}).text.strip("\n")))
            list_score.append((match.find("td",{"cell-bonus-b"}).text.strip("\n")))
            match_list = [journee_name[journee_n]] +list_info + list_score
            saison.append(match_list)

        for match in container[0].findAll("tr",{"class": 'info-line after table-hr'}):
            list_info = [i.text for i in (match.findAll("span",{"format-full"}))[:-1]]
            list_score = match.find("td",{"cell-score"}).text.strip().split("-")
            list_score.append((match.find("td",{"cell-bonus-a"}).text.strip("\n")))
            list_score.append((match.find("td",{"cell-bonus-b"}).text.strip("\n")))
            match_list = [journee_name[journee_n]] +list_info + list_score
            saison.append(match_list)
        journee_n +=1


In [25]:
cols = ["saison - journee", "date","equipe_d", "equipe_e", "score_d", "score_e", "bonus_d", "bonus_e"]
data = pd.DataFrame(saison,columns=cols)

In [26]:
data.to_csv("data_saison.csv")

In [39]:
data["score_d"] = data["score_d"].astype(int)
# data.groupby("equipe_d").agg({"score_d": "sum"}).sort_values(by = "s")

### Classement

In [86]:
classement = []
wiki_temp = [x["href"] for x in soup_class.find("div", {"class": "tabs-content"}).findAll("a", {"class": "filter"})[:10]]

for w in wiki_temp:
    requete_class = requests.get(fixe + w)
    page_class = requete_class.content
    soup_class = b(page_class)

    wiki_temp_class = []
    journee_name_class = []
    for i in soup_class.find("div", {"class": "tabs-content"}).findAll("a", {"class": "filter"})[10:]:
    #     i.findAll("span", {"class": "field-content"})[15:]
        wiki_temp_class.append(i["href"])
        journee_name_class.append(i["data-title"])
    
    
    p = 1

    for k,j in enumerate(wiki_temp_class):
        requete = requests.get(fixe+j)
        page = requete.content
        soup_class = b(page)
        for x in range(14):
            try:
                team = []
                team.append(p)
                team.append(journee_name_class[k])

                team.append(soup_class.find("div", {"class": "scroll-wrapper"}).findAll("td", {"class": "views-field views-field-field-ranking"})[x].text.strip())
                team.append(soup_class.find("div", {"class": "scroll-wrapper"}).findAll("td", {"class": "views-field-field-points"})[x].text.strip())
                team.append(soup_class.find("div", {"class": "scroll-wrapper"}).findAll("td", {"class": "views-field views-field-field--quipe"})[x].text.strip())
                team.append(soup_class.find("div", {"class": "scroll-wrapper"}).findAll("td", {"class": "views-field-field-nbmatchsplayed"})[x].text.strip())
                team.append(soup_class.find("div", {"class": "scroll-wrapper"}).findAll("td", {"class": "views-field-field-won"})[x].text.strip())
                team.append(soup_class.find("div", {"class": "scroll-wrapper"}).findAll("td", {"class": "views-field-field-draws"})[x].text.strip())
                team.append(soup_class.find("div", {"class": "scroll-wrapper"}).findAll("td", {"class": "views-field-field-lost"})[x].text.strip())
                team.append(soup_class.find("div", {"class": "scroll-wrapper"}).findAll("td", {"class": "views-field-field-bonus"})[x].text.strip())
                team.append(soup_class.find("div", {"class": "scroll-wrapper"}).findAll("td", {"class": "views-field-field-pointsscored"})[x].text.strip())
                team.append(soup_class.find("div", {"class": "scroll-wrapper"}).findAll("td", {"class": "views-field-field-pointsconceded"})[x].text.strip())

                team.append(soup_class.find("div", {"class": "scroll-wrapper"}).findAll("td", {"class": "views-field-field-diff"})[x].text.strip()[1:])


                classement.append(team)
            except:
                None
        p+=1

pas la bonne taille
pas la bonne taille
pas la bonne taille
pas la bonne taille
pas la bonne taille
pas la bonne taille
pas la bonne taille
pas la bonne taille
pas la bonne taille
pas la bonne taille
pas la bonne taille
pas la bonne taille
pas la bonne taille
pas la bonne taille
pas la bonne taille
pas la bonne taille
pas la bonne taille
pas la bonne taille
pas la bonne taille
pas la bonne taille
pas la bonne taille
pas la bonne taille
pas la bonne taille
pas la bonne taille
pas la bonne taille
pas la bonne taille
pas la bonne taille
pas la bonne taille
pas la bonne taille
pas la bonne taille
pas la bonne taille
pas la bonne taille
pas la bonne taille
pas la bonne taille
pas la bonne taille
pas la bonne taille
pas la bonne taille
pas la bonne taille
pas la bonne taille
pas la bonne taille
pas la bonne taille
pas la bonne taille
pas la bonne taille
pas la bonne taille
pas la bonne taille
pas la bonne taille
pas la bonne taille
pas la bonne taille
pas la bonne taille
pas la bonne taille


In [87]:
col_class = ["journee","journee_nom", "classement", "nb_pts","equipe","nb_matchs_joues","victoire","nul", "defaite", "bonus", "pts_marques","pts_pris","ga"]
pd.DataFrame(classement,columns = col_class).to_csv("data_classement.csv")